## Code taken from : Tech-At-Work in this video [Easily Fine Tune ChatGPT 3.5 to Outperform GPT-4!](https://youtu.be/8Ieu2v0v4oc?si=LLlZB0arElYWJWVH) (accessed last 20.11.2023)

In [1]:
!pip install openai
!pip install numpy
!pip install tiktoken


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 6.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import openai
import csv
import json
import os
import numpy as np
from collections import defaultdict
import tiktoken

In [2]:
# please replace this string with the actual api key
openai.api_key = "API-KEY"

In [4]:
#Load CSV data in for train_50.csv
csv_file_path = '../random-training-data-other-taggers/csv/train_50.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = 'train50-cleaned.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

In [3]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = 'train50-cleaned.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")


Num examples: 200
First example:
{'role': 'system', 'content': 'You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other.'}
{'role': 'user', 'content': "Return the UPOS tag for the tokens of the sentence: '  'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."}
{'role': 'assistant', 'content': '\\t\\n'}
No errors found


In [3]:
# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")



In [6]:
# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 50000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_1k_tokens = 0.0080  # Cost for every 1000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 1000) * cost_per_1k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 658
mean / median: 272.82, 249.5
p5 / p95: 173.8, 407.5

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 409
mean / median: 104.32, 86.5
p5 / p95: 27.9, 204.60000000000002

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning
Dataset has ~13641 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~40923 tokens
Estimated cost for fine-tuning: approximately $0.33


In [4]:
# Function to save the dataset as a JSONL file
def save_to_jsonl(conversations, file_path):
    with open(file_path, 'w') as file:
        for conversation in conversations:
            json_line = json.dumps(conversation)
            file.write(json_line + '\n')

In [9]:
# Specify the path where you want to save the JSONL file
jsonl_file_path = 'train50-cleaned.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [6]:
#Upload data for training
training_file_name = 'train50-cleaned.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

Training file id: file-hmZVvME5IHk7zJ7TlSXNXDFk


In [7]:
#Create Fine-Tuning Job
suffix_name = "train-50"

response = openai.FineTuningJob.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-LjWWFBZ2LVJMcErWp3BOVwoz",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1698946835,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-hmZVvME5IHk7zJ7TlSXNXDFk",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}


In [23]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tuning job: ftjob-LjWWFBZ2LVJMcErWp3BOVwoz
Validating training file: file-hmZVvME5IHk7zJ7TlSXNXDFk
Files validated, moving job to queued state
Fine-tuning job started
Step 1/150: training loss=0.62
Step 11/150: training loss=0.09
Step 21/150: training loss=0.06
Step 31/150: training loss=0.13
Step 41/150: training loss=0.05
Step 51/150: training loss=0.01
Step 61/150: training loss=0.08
Step 71/150: training loss=0.01
Step 81/150: training loss=0.21
Step 91/150: training loss=0.00
Step 101/150: training loss=0.00
Step 111/150: training loss=0.03
Step 121/150: training loss=0.01
Step 131/150: training loss=0.06
Step 141/150: training loss=0.00
New fine-tuned model created: ft:gpt-3.5-turbo-0613:cl-uzh:train-50:8GW6NJ4E


In [24]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

{
  "object": "fine_tuning.job",
  "id": "ftjob-LjWWFBZ2LVJMcErWp3BOVwoz",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1698946835,
  "finished_at": 1698947338,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:cl-uzh:train-50:8GW6NJ4E",
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [
    "file-CuG5MK0AFXkZOkLnf93ATNWG"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-hmZVvME5IHk7zJ7TlSXNXDFk",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 40623,
  "error": null
}

Fine-tuned model id: ft:gpt-3.5-turbo-0613:cl-uzh:train-50:8GW6NJ4E


## Now for model train_100

In [25]:
#Load CSV data in for train_100.csv
csv_file_path = '../random-training-data-other-taggers/csv/train_100.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = 'train100-cleaned.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

In [26]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = 'train100-cleaned.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 50000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_1k_tokens = 0.0080  # Cost for every 1000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 1000) * cost_per_1k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing


Num examples: 100
First example:
{'role': 'system', 'content': 'You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other.'}
{'role': 'user', 'content': "Return the UPOS tag for the tokens of the sentence: '  'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."}
{'role': 'assistant', 'content': '\\t\\n'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 658
mean / median: 278.06, 249.5
p5 / p95: 178.7, 412.70000000000005

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 409
mean / median: 108.27, 87.0
p5

In [27]:
# Specify the path where you want to save the JSONL file
jsonl_file_path = 'train100-cleaned.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [28]:
#Upload data for training
training_file_name = 'train100-cleaned.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

####################

#Create Fine-Tuning Job
suffix_name = "train-100"

response = openai.FineTuningJob.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)


Training file id: file-0LNOT5KHjfF74FdNgo8Mr8CY
{
  "object": "fine_tuning.job",
  "id": "ftjob-cwJaUkv3rT6oI2vxwfSJXLr8",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1698947569,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-0LNOT5KHjfF74FdNgo8Mr8CY",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}


In [58]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tuning job: ftjob-cwJaUkv3rT6oI2vxwfSJXLr8
Validating training file: file-0LNOT5KHjfF74FdNgo8Mr8CY
Files validated, moving job to queued state
Fine-tuning job started
Step 1/300: training loss=0.17
Step 11/300: training loss=0.20
Step 21/300: training loss=0.24
Step 31/300: training loss=0.05
Step 41/300: training loss=0.03
Step 51/300: training loss=0.02
Step 61/300: training loss=0.07
Step 71/300: training loss=0.04
Step 81/300: training loss=0.09
Step 91/300: training loss=0.00
Step 101/300: training loss=0.00
Step 111/300: training loss=0.02
Step 121/300: training loss=0.04
Step 131/300: training loss=0.00
Step 141/300: training loss=0.00
Step 151/300: training loss=0.02
Step 161/300: training loss=0.00
Step 171/300: training loss=0.11
Step 181/300: training loss=0.11
Step 191/300: training loss=0.05
Step 201/300: training loss=0.00
Step 211/300: training loss=0.00
Step 221/300: training loss=0.14
Step 231/300: training loss=0.00
Step 241/300: training loss=0.11
Step 2

In [59]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

{
  "object": "fine_tuning.job",
  "id": "ftjob-cwJaUkv3rT6oI2vxwfSJXLr8",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1698947569,
  "finished_at": 1698948351,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:cl-uzh:train-100:8GWMiGKN",
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [
    "file-l8tSmDlLxhqFQ0fddSpzjy1A"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-0LNOT5KHjfF74FdNgo8Mr8CY",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 82818,
  "error": null
}

Fine-tuned model id: ft:gpt-3.5-turbo-0613:cl-uzh:train-100:8GWMiGKN


## Now for model train_200¶

In [3]:
#Load CSV data in for train_200.csv
csv_file_path = '../random-training-data-other-taggers/csv/train_200.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = 'train200-cleaned.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

In [7]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = 'train200-cleaned.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")


# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 50000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_1k_tokens = 0.0080  # Cost for every 1000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 1000) * cost_per_1k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing


Num examples: 200
First example:
{'role': 'system', 'content': 'You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other.'}
{'role': 'user', 'content': "Return the UPOS tag for the tokens of the sentence: '  'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."}
{'role': 'assistant', 'content': '\\t\\n'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 689
mean / median: 276.25, 249.0
p5 / p95: 180.8, 406.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 417
mean / median: 106.905, 86.0
p5 / p95: 34.0

In [11]:
# Specify the path where you want to save the JSONL file
jsonl_file_path = 'train200-cleaned.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [12]:
#Upload data for training
training_file_name = 'train200-cleaned.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

Training file id: file-p4gs91wol87czouFTwDaKRYy


In [13]:
#Create Fine-Tuning Job
suffix_name = "train-200"

response = openai.FineTuningJob.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)


{
  "object": "fine_tuning.job",
  "id": "ftjob-bPm1nWW1CmDNh5ihe8PIQeYQ",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699126937,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-p4gs91wol87czouFTwDaKRYy",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}


In [36]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Step 121/600: training loss=0.01
Step 131/600: training loss=0.34
Step 141/600: training loss=0.03
Step 151/600: training loss=0.04
Step 161/600: training loss=0.01
Step 171/600: training loss=0.06
Step 181/600: training loss=0.00
Step 191/600: training loss=0.00
Step 201/600: training loss=0.08
Step 211/600: training loss=0.00
Step 221/600: training loss=0.00
Step 231/600: training loss=0.06
Step 241/600: training loss=0.04
Step 251/600: training loss=0.00
Step 261/600: training loss=0.02
Step 271/600: training loss=0.00
Step 281/600: training loss=0.00
Step 291/600: training loss=0.00
Step 301/600: training loss=0.03
Step 311/600: training loss=0.00
Step 321/600: training loss=0.26
Step 331/600: training loss=0.06
Step 341/600: training loss=0.00
Step 351/600: training loss=0.04
Step 361/600: training loss=0.06
Step 371/600: training loss=0.15
Step 381/600: training loss=0.00
Step 391/600: training loss=0.00
Step 401/600: training loss=0.08
Step 411/600: training loss=0.07
Step 421/6

In [37]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)


{
  "object": "fine_tuning.job",
  "id": "ftjob-bPm1nWW1CmDNh5ihe8PIQeYQ",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699126937,
  "finished_at": 1699128260,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:cl-uzh:train-200:8HHAT5rX",
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [
    "file-nvfWh0w0rLlL461KIJeEA9EK"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-p4gs91wol87czouFTwDaKRYy",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 164550,
  "error": null
}

Fine-tuned model id: ft:gpt-3.5-turbo-0613:cl-uzh:train-200:8HHAT5rX


## Now for model train_500¶

In [5]:
#Load CSV data in for train_500.csv
csv_file_path = '../random-training-data-other-taggers/csv/train_500.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = 'train500-cleaned.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

In [6]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = 'train500-cleaned.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")


# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 50000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_1k_tokens = 0.0080  # Cost for every 1000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 1000) * cost_per_1k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing


Num examples: 500
First example:
{'role': 'system', 'content': 'You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other.'}
{'role': 'user', 'content': "Return the UPOS tag for the tokens of the sentence: '  'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."}
{'role': 'assistant', 'content': '\\t\\n'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 853
mean / median: 278.11, 248.0
p5 / p95: 177.9, 401.20000000000005

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 553
mean / median: 108.296, 86.0
p

In [7]:
# Specify the path where you want to save the JSONL file
jsonl_file_path = 'train500-cleaned.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [8]:
#Upload data for training
training_file_name = 'train500-cleaned.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

Training file id: file-nN3js1LWWjl7YBTM5NXDNYOd


In [52]:
#Create Fine-Tuning Job
suffix_name = "train-500"

response = openai.FineTuningJob.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)


RateLimitError: This fine-tune request has been rate-limited. Your organization has reached the maximum of 3 active requests (3 running, 0 pending) for the model 'gpt-3.5-turbo-0613'.

In [5]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def create_fine_tuning_job(training_file_id):
    response = openai.FineTuningJob.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
    )
    return response["id"]


In [9]:
#Create Fine-Tuning Job
suffix_name = "train-500"

response = create_fine_tuning_job(training_file_id)
job_id = response

print(response)

ftjob-UrrnRGBYYD3kKkhhoVzrgtdY


In [33]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tuning job: ftjob-UrrnRGBYYD3kKkhhoVzrgtdY
Validating training file: file-nN3js1LWWjl7YBTM5NXDNYOd
Files validated, moving job to queued state
Fine-tuning job started
Step 1/1500: training loss=0.35
Step 101/1500: training loss=0.04
Step 201/1500: training loss=0.13
Step 301/1500: training loss=0.16
Step 401/1500: training loss=0.04
Step 501/1500: training loss=0.00
Step 601/1500: training loss=0.00
Step 701/1500: training loss=0.00
Step 801/1500: training loss=0.00
Step 901/1500: training loss=0.13
Step 1001/1500: training loss=0.00
Step 1101/1500: training loss=0.00
Step 1201/1500: training loss=0.00
Step 1301/1500: training loss=0.00
Step 1401/1500: training loss=0.45
New fine-tuned model created: ft:gpt-3.5-turbo-0613:cl-uzh:train-500:8HSFR0js
The job has successfully completed


In [34]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)


{
  "object": "fine_tuning.job",
  "id": "ftjob-UrrnRGBYYD3kKkhhoVzrgtdY",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699168059,
  "finished_at": 1699170852,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:cl-uzh:train-500:8HSFR0js",
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [
    "file-O5a8MgzVpvtnDOHR737EBupu"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-nN3js1LWWjl7YBTM5NXDNYOd",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 414165,
  "error": null
}

Fine-tuned model id: ft:gpt-3.5-turbo-0613:cl-uzh:train-500:8HSFR0js


## Now for model train_1000¶

In [6]:
#Load CSV data in for train_1000.csv
csv_file_path = '../random-training-data-other-taggers/csv/train_1000.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = 'train1000-cleaned.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

In [7]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = 'train1000-cleaned.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")


# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 50000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_1k_tokens = 0.0080  # Cost for every 1000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 1000) * cost_per_1k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing


Num examples: 1000
First example:
{'role': 'system', 'content': 'You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other.'}
{'role': 'user', 'content': "Return the UPOS tag for the tokens of the sentence: '  'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."}
{'role': 'assistant', 'content': '\\t\\n'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 1112
mean / median: 273.239, 245.0
p5 / p95: 179.0, 392.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 752
mean / median: 104.591, 83.0
p5 / p95: 3

In [8]:
# Specify the path where you want to save the JSONL file
jsonl_file_path = 'train1000-cleaned.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [9]:
#Upload data for training
training_file_name = 'train1000-cleaned.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

Training file id: file-RuKGfyKAbOUNlSUbiu8Moom0


In [11]:
#Create Fine-Tuning Job

suffix_name = "train-1000"

response = create_fine_tuning_job(training_file_id)

job_id = response

print(response)


ftjob-ZFTuIn2nlY1BAIZ6rDpgKxtp


In [58]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tuning job: ftjob-ZFTuIn2nlY1BAIZ6rDpgKxtp
Validating training file: file-RuKGfyKAbOUNlSUbiu8Moom0
Files validated, moving job to queued state
Fine-tuning job started
Step 1/1500: training loss=0.27
Step 101/1500: training loss=0.11
Step 201/1500: training loss=0.04
Step 301/1500: training loss=0.09
Step 401/1500: training loss=0.00
Step 501/1500: training loss=0.00
Step 601/1500: training loss=0.02
Step 701/1500: training loss=0.03
Step 801/1500: training loss=0.01
Step 901/1500: training loss=0.04
Step 1001/1500: training loss=0.01
Step 1101/1500: training loss=0.00
Step 1201/1500: training loss=0.00
Step 1301/1500: training loss=0.00
Step 1401/1500: training loss=0.00
New fine-tuned model created: ft:gpt-3.5-turbo-0613:cl-uzh:train-1000:8HUHOHgt
The job has successfully completed


In [59]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)


{
  "object": "fine_tuning.job",
  "id": "ftjob-ZFTuIn2nlY1BAIZ6rDpgKxtp",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699175903,
  "finished_at": 1699178661,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:cl-uzh:train-1000:8HUHOHgt",
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [
    "file-BHUNWT1F2RLbn2KtATQUZAMW"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-RuKGfyKAbOUNlSUbiu8Moom0",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 813717,
  "error": null
}

Fine-tuned model id: ft:gpt-3.5-turbo-0613:cl-uzh:train-1000:8HUHOHgt


## Now for model train_2000¶

In [6]:
#Load CSV data in for train_2000.csv
csv_file_path = '../random-training-data-other-taggers/csv/train_2000.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = 'train2000-cleaned.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

In [7]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = 'train2000-cleaned.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")


# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 50000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_1k_tokens = 0.0080  # Cost for every 1000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 1000) * cost_per_1k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing


Num examples: 2000
First example:
{'role': 'system', 'content': 'You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other.'}
{'role': 'user', 'content': "Return the UPOS tag for the tokens of the sentence: '  'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."}
{'role': 'assistant', 'content': '\\t\\n'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 1112
mean / median: 276.684, 245.0
p5 / p95: 178.0, 406.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 752
mean / median: 107.282, 83.0
p5 / p95: 3

In [8]:
# Specify the path where you want to save the JSONL file
jsonl_file_path = 'train2000-cleaned.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [9]:
#Upload data for training
training_file_name = 'train2000-cleaned.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

Training file id: file-zSGfzNRwUqGmGzgcggScrxPL


In [10]:
#Create Fine-Tuning Job
suffix_name = "train-2000"

response = create_fine_tuning_job(training_file_id)

job_id = response

print(response)


ftjob-cbYY7toGBQ8qWoJITmPs5sKF


In [39]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tuning job: ftjob-cbYY7toGBQ8qWoJITmPs5sKF
Validating training file: file-zSGfzNRwUqGmGzgcggScrxPL
Files validated, moving job to queued state
Fine-tuning job started
Step 1/1500: training loss=0.26
Step 101/1500: training loss=0.07
Step 201/1500: training loss=0.05
Step 301/1500: training loss=0.01
Step 401/1500: training loss=0.00
Step 501/1500: training loss=0.00
Step 601/1500: training loss=0.02
Step 701/1500: training loss=0.00
Step 801/1500: training loss=0.04
Step 901/1500: training loss=0.01
Step 1001/1500: training loss=0.02
Step 1101/1500: training loss=0.05
Step 1201/1500: training loss=0.00
Step 1301/1500: training loss=0.07
Step 1401/1500: training loss=0.00
New fine-tuned model created: ft:gpt-3.5-turbo-0613:cl-uzh:train-2000:8HXS81Fi
The job has successfully completed


In [40]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)


{
  "object": "fine_tuning.job",
  "id": "ftjob-cbYY7toGBQ8qWoJITmPs5sKF",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699187639,
  "finished_at": 1699190859,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:cl-uzh:train-2000:8HXS81Fi",
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [
    "file-e8kzJ2Em4QvJLCSzwygZfE5k"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-zSGfzNRwUqGmGzgcggScrxPL",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 1648104,
  "error": null
}

Fine-tuned model id: ft:gpt-3.5-turbo-0613:cl-uzh:train-2000:8HXS81Fi


## Now for model train_5000¶

In [6]:
#Load CSV data in for train_5000.csv
csv_file_path = '../random-training-data-other-taggers/csv/train_5000.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = 'train5000-cleaned.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

In [7]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = 'train5000-cleaned.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")


# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 50000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_1k_tokens = 0.0080  # Cost for every 1000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 1000) * cost_per_1k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing


Num examples: 5000
First example:
{'role': 'system', 'content': 'You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other.'}
{'role': 'user', 'content': "Return the UPOS tag for the tokens of the sentence: '  'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."}
{'role': 'assistant', 'content': '\\t\\n'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 1545
mean / median: 275.6108, 243.0
p5 / p95: 179.0, 400.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 1067
mean / median: 106.4704, 82.0
p5 / p95

In [8]:
# Specify the path where you want to save the JSONL file
jsonl_file_path = 'train5000-cleaned.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [9]:
#Upload data for training
training_file_name = 'train5000-cleaned.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

Training file id: file-Yq46yBwRn2R4szfpsrUyvHkT


In [10]:
#Create Fine-Tuning Job
suffix_name = "train-5000"

response = create_fine_tuning_job(training_file_id)

job_id = response

print(response)


ftjob-7DZF4DkExn5QmRd0ohFyEAS0


In [26]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tuning job: ftjob-7DZF4DkExn5QmRd0ohFyEAS0
Validating training file: file-Yq46yBwRn2R4szfpsrUyvHkT
Files validated, moving job to queued state
Fine-tuning job started
Step 1/1500: training loss=0.27
Step 101/1500: training loss=0.03
Step 201/1500: training loss=0.03
Step 301/1500: training loss=0.02
Step 401/1500: training loss=0.01
Step 501/1500: training loss=0.02
Step 601/1500: training loss=0.01
Step 701/1500: training loss=0.01
Step 801/1500: training loss=0.01
Step 901/1500: training loss=0.02
Step 1001/1500: training loss=0.00
Step 1101/1500: training loss=0.02
Step 1201/1500: training loss=0.00
Step 1301/1500: training loss=0.01
Step 1401/1500: training loss=0.00
New fine-tuned model created: ft:gpt-3.5-turbo-0613:cl-uzh:train-5000:8HqJwp87
The job has successfully completed


In [27]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)


{
  "object": "fine_tuning.job",
  "id": "ftjob-7DZF4DkExn5QmRd0ohFyEAS0",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699260899,
  "finished_at": 1699263387,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:cl-uzh:train-5000:8HqJwp87",
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [
    "file-mRqxe4ZJuINr88FQjMbBob8z"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-Yq46yBwRn2R4szfpsrUyvHkT",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 4104162,
  "error": null
}

Fine-tuned model id: ft:gpt-3.5-turbo-0613:cl-uzh:train-5000:8HqJwp87


## Now for model train_10000¶

In [6]:
#Load CSV data in for train_10000.csv
csv_file_path = '../random-training-data-other-taggers/csv/train_10000.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = 'train10000-cleaned.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

In [7]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = 'train10000-cleaned.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")


# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 50000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_1k_tokens = 0.0080  # Cost for every 1000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 1000) * cost_per_1k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing


Num examples: 10000
First example:
{'role': 'system', 'content': 'You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other.'}
{'role': 'user', 'content': "Return the UPOS tag for the tokens of the sentence: '  'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."}
{'role': 'assistant', 'content': '\\t\\n'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 2140
mean / median: 276.9378, 243.0
p5 / p95: 179.0, 403.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 1547
mean / median: 107.4908, 82.0
p5 / p9

In [8]:
# Specify the path where you want to save the JSONL file
jsonl_file_path = 'train10000-cleaned.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [9]:
#Upload data for training
training_file_name = 'train10000-cleaned.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

Training file id: file-YE4ZzTzKMNXqZYiT3lH1HA5D


In [10]:
#Create Fine-Tuning Job
suffix_name = "train-10000"

response = create_fine_tuning_job(training_file_id)

job_id = response

print(response)


ftjob-ErsMkmBfoVtMAzMcFQTj3M0k


In [50]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tuning job: ftjob-ErsMkmBfoVtMAzMcFQTj3M0k
Validating training file: file-YE4ZzTzKMNXqZYiT3lH1HA5D
Files validated, moving job to queued state
Fine-tuning job started
Step 1/1539: training loss=0.22
Step 101/1539: training loss=0.02
Step 201/1539: training loss=0.01
Step 301/1539: training loss=0.01
Step 401/1539: training loss=0.02
Step 501/1539: training loss=0.01
Step 601/1539: training loss=0.03
Step 701/1539: training loss=0.01
Step 801/1539: training loss=0.02
Step 901/1539: training loss=0.01
Step 1001/1539: training loss=0.02
Step 1101/1539: training loss=0.01
Step 1201/1539: training loss=0.00
Step 1301/1539: training loss=0.01
Step 1401/1539: training loss=0.01
Step 1501/1539: training loss=0.01
New fine-tuned model created: ft:gpt-3.5-turbo-0613:cl-uzh:train-10000:8HrmXwL4
The job has successfully completed


In [51]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)


{
  "object": "fine_tuning.job",
  "id": "ftjob-ErsMkmBfoVtMAzMcFQTj3M0k",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699264870,
  "finished_at": 1699269004,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:cl-uzh:train-10000:8HrmXwL4",
  "organization_id": "org-HjWBseLU0iDzg4cx01nO6JrY",
  "result_files": [
    "file-3PePRXGS3RUZGbxWHf6srrS8"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-YE4ZzTzKMNXqZYiT3lH1HA5D",
  "hyperparameters": {
    "n_epochs": 2
  },
  "trained_tokens": 5498756,
  "error": null
}

Fine-tuned model id: ft:gpt-3.5-turbo-0613:cl-uzh:train-10000:8HrmXwL4


## Now for model train_20000¶

In [34]:
#Load CSV data in for train_20000.csv
csv_file_path = '../random-training-data-other-taggers/csv/train_20000.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = 'train20000-cleaned.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

In [35]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = 'train20000-cleaned.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")


# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 50000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_1k_tokens = 0.0080  # Cost for every 1000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 1000) * cost_per_1k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing


Num examples: 20000
First example:
{'role': 'system', 'content': 'You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other.'}
{'role': 'user', 'content': "Return the UPOS tag for the tokens of the sentence: '  'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."}
{'role': 'assistant', 'content': '\\t\\n'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 2241
mean / median: 275.7084, 243.0
p5 / p95: 179.0, 401.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 1621
mean / median: 106.55645, 82.0
p5 / p

In [ ]:
# Specify the path where you want to save the JSONL file
jsonl_file_path = 'train20000-cleaned.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [ ]:
#Upload data for training
training_file_name = 'train20000-cleaned.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

In [ ]:
#Create Fine-Tuning Job
suffix_name = "train-20000"

response = create_fine_tuning_job(training_file_id)

job_id = response

print(response)


In [ ]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

In [ ]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)


## Now for model train_total

In [36]:
#Load CSV data in for train_total.csv
csv_file_path = '../random-training-data-other-taggers/csv/train_total.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = 'train_total-cleaned.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

JSON decode error for cell '{"messages":[{"role":"system","content":"You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other."},{"role":"user","content":"Return the UPOS tag for the tokens of the sentence: ' Qui postquam tecta tyranni intravere sui , " non est hortamine longo nunc " ait " utendum . . 'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."},{"role":"assistant","content":"Qui\\tPRON\\n postquam\\tCONJ\\n tecta\\tNOUN\\n tyranni\\tNOUN\\n intravere\\tVERB\\n sui\\tADJ\\n ,\\tPUNCT\\n "\\tPUNCT\\n non\\tADV\\n est\\tVERB\\n hortamine\\tNOUN\\n longo\\tADJ\\n nunc\\tADV\\n "\\tPUNCT\\n ait\\tVERB\\n "\\tPUNCT\\n utendum\\tVERB\\n .\\tPUNCT\\n .\\tPUNCT\\n"}]}': Expecting ',' delimiter: line 1 column 399 (char 398)


In [37]:
#from OpenAI website to format data;  https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

# Next, we specify the data path and open the JSONL file

data_path = 'train_total-cleaned.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")


# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 50000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_1k_tokens = 0.0080  # Cost for every 1000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 1000) * cost_per_1k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing


Num examples: 43866
First example:
{'role': 'system', 'content': 'You are a Latin linguist and part-of-speech tagging expert. You are using UPOS (universal part of speech tags). UPOS tags are ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB and X. X stands for other.'}
{'role': 'user', 'content': "Return the UPOS tag for the tokens of the sentence: '  'Return the tags in the format TOKEN\tTag\n. Only output the token and the tag (no explanations, no translations, no additional text)."}
{'role': 'assistant', 'content': '\\t\\n'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 2846
mean / median: 276.54060092098666, 243.0
p5 / p95: 179.0, 403.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 2089
mean / median: 107.19965349

In [ ]:
# Specify the path where you want to save the JSONL file
jsonl_file_path = 'train_total-cleaned.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [ ]:
#Upload data for training
training_file_name = 'train_total-cleaned.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

In [ ]:
#Create Fine-Tuning Job
suffix_name = "train-total"

response = create_fine_tuning_job(training_file_id)

job_id = response

print(response)


In [ ]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

In [ ]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)
